In [1]:
import os
from typing import List, Optional, Tuple

from huggingface_hub import snapshot_download

from vllm import EngineArgs, LLMEngine, RequestOutput, SamplingParams
from vllm.lora.request import LoRARequest

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

HF_TOKEN = os.getenv("HF_TOKEN")
WANDB_API_KEY = os.getenv("WANDB_API_KEY")


In [2]:
def create_test_prompts(
        sql_lora_path: str,
        oasst_lora_path: str
) -> List[Tuple[str, SamplingParams, Optional[LoRARequest]]]:
    """Create a list of test prompts with their sampling parameters.

    2 requests for base model, 4 requests for the LoRA. We define 2
    different LoRA adapters (using the same model for demo purposes).
    Since we also set `max_loras=1`, the expectation is that the requests
    with the second LoRA adapter will be ran after all requests with the
    first adapter have finished.
    """
    return [
        ("A robot may not injure a human being",
         SamplingParams(temperature=0.0,
                        logprobs=1,
                        prompt_logprobs=1,
                        max_tokens=128), None),
        ("To be or not to be,",
         SamplingParams(temperature=0.8,
                        top_k=5,
                        presence_penalty=0.2,
                        max_tokens=128), None),
        (
            "[user] Write a SQL query to answer the question based on the table schema.\n\n context: CREATE TABLE table_name_74 (icao VARCHAR, airport VARCHAR)\n\n question: Name the ICAO for lilongwe international airport [/user] [assistant]",  # noqa: E501
            SamplingParams(temperature=0.0,
                           logprobs=1,
                           prompt_logprobs=1,
                           max_tokens=128,
                           stop_token_ids=[32003]),
            LoRARequest("sql-lora", 1, sql_lora_path)),
        (
            "[user] Write a SQL query to answer the question based on the table schema.\n\n context: CREATE TABLE table_name_11 (nationality VARCHAR, elector VARCHAR)\n\n question: When Anchero Pantaleone was the elector what is under nationality? [/user] [assistant]",  # noqa: E501
            SamplingParams(n=3,
                           best_of=3,
                           use_beam_search=True,
                           temperature=0,
                           max_tokens=128,
                           stop_token_ids=[32003]),
            LoRARequest("sql-lora", 1, sql_lora_path)),
        (
            "[user] Write a SQL query to answer the question based on the table schema.\n\n context: CREATE TABLE table_name_74 (icao VARCHAR, airport VARCHAR)\n\n question: Name the ICAO for lilongwe international airport [/user] [assistant]",  # noqa: E501
            SamplingParams(temperature=0.0,
                           logprobs=1,
                           prompt_logprobs=1,
                           max_tokens=128,
                           stop_token_ids=[32003]),
            LoRARequest("sql-lora2", 2, sql_lora_path)),
        (
            "[user] Write a SQL query to answer the question based on the table schema.\n\n context: CREATE TABLE table_name_11 (nationality VARCHAR, elector VARCHAR)\n\n question: When Anchero Pantaleone was the elector what is under nationality? [/user] [assistant]",  # noqa: E501
            SamplingParams(n=3,
                           best_of=3,
                           use_beam_search=True,
                           temperature=0,
                           max_tokens=128,
                           stop_token_ids=[32003]),
            LoRARequest("sql-lora", 1, sql_lora_path)),
        (
            "[user] Can you explain the difference between a river and a riverboat? [/user] [assistant]",  # noqa: E501
            SamplingParams(temperature=0.0,
                           logprobs=1,
                           prompt_logprobs=1,
                           max_tokens=128,
                           stop_token_ids=[32003]),
            LoRARequest("oasst-lora", 1, oasst_lora_path)),
    ]

In [3]:
def process_requests(engine: LLMEngine,
                     test_prompts: List[Tuple[str, SamplingParams,
                                              Optional[LoRARequest]]]):
    """Continuously process a list of prompts and handle the outputs."""
    request_id = 0

    while test_prompts or engine.has_unfinished_requests():
        if test_prompts:
            prompt, sampling_params, lora_request = test_prompts.pop(0)
            engine.add_request(str(request_id),
                               prompt,
                               sampling_params,
                               lora_request=lora_request)
            request_id += 1

        request_outputs: List[RequestOutput] = engine.step()

        for request_output in request_outputs:
            if request_output.finished:
                print(request_output)
    return request_outputs

In [4]:
def initialize_engine() -> LLMEngine:
    """Initialize the LLMEngine."""
    # max_loras: controls the number of LoRAs that can be used in the same
    #   batch. Larger numbers will cause higher memory usage, as each LoRA
    #   slot requires its own preallocated tensor.
    # max_lora_rank: controls the maximum supported rank of all LoRAs. Larger
    #   numbers will cause higher memory usage. If you know that all LoRAs will
    #   use the same rank, it is recommended to set this as low as possible.
    # max_cpu_loras: controls the size of the CPU LoRA cache.
    engine_args = EngineArgs(model="meta-llama/Llama-2-7b-hf",
                             enable_lora=True,
                             max_loras=2,
                             max_lora_rank=8,
                             max_cpu_loras=2,
                             max_num_seqs=256,
                             tensor_parallel_size=1,)
    return LLMEngine.from_engine_args(engine_args)

In [5]:

def main():
    """Main function that sets up and runs the prompt processing."""
    engine = initialize_engine()
    sql_lora_path = snapshot_download(repo_id="yard1/llama-2-7b-sql-lora-test")
    oasst_lora_path = snapshot_download(repo_id="kaitchup/Llama-2-7B-oasstguanaco-adapter-1e")
    test_prompts = create_test_prompts(sql_lora_path, oasst_lora_path)
    output = process_requests(engine, test_prompts)
    return output


# if __name__ == '__main__':
#     main()

In [6]:
test = main()


INFO 06-02 13:50:38 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='meta-llama/Llama-2-7b-hf', speculative_config=None, tokenizer='meta-llama/Llama-2-7b-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Llama-2-7b-hf)


/home/harpreet_guest2/akshat/Multi-Qlora/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 06-02 13:50:38 weight_utils.py:207] Using model weights format ['*.safetensors']
INFO 06-02 13:50:40 model_runner.py:146] Loading model weights took 12.5581 GB
INFO 06-02 13:50:42 gpu_executor.py:83] # GPU blocks: 3781, # CPU blocks: 512
INFO 06-02 13:50:44 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-02 13:50:44 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-02 13:50:47 model_runner.py:924] Graph capturing finished in 4 secs.


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

RequestOutput(request_id=6, prompt='[user] Can you explain the difference between a river and a riverboat? [/user] [assistant]', prompt_token_ids=[1, 32000, 29871, 1815, 366, 5649, 278, 4328, 1546, 263, 8580, 322, 263, 8580, 833, 271, 29973, 29871, 32001, 259, 32002], prompt_logprobs=[None, {32000: Logprob(logprob=-0.004840085748583078, rank=1, decoded_token='[user]')}, {29871: Logprob(logprob=-0.1254420429468155, rank=1, decoded_token='')}, {1815: Logprob(logprob=-15.57059383392334, rank=179, decoded_token='Can'), 14350: Logprob(logprob=-0.0002809368306770921, rank=1, decoded_token='Write')}, {366: Logprob(logprob=-1.1787023544311523, rank=1, decoded_token='you')}, {5649: Logprob(logprob=-3.265174388885498, rank=4, decoded_token='explain'), 2649: Logprob(logprob=-1.858924388885498, rank=1, decoded_token='tell')}, {278: Logprob(logprob=-1.4558932781219482, rank=1, decoded_token='the')}, {4328: Logprob(logprob=-2.698925018310547, rank=1, decoded_token='difference')}, {1546: Logprob(logp

In [7]:
len(test)

2

In [25]:
for elem in test:
    print("Prompt:", elem.prompt)
    for output in elem.outputs:
        print("Output:", output.text)
    print()  # empty line to separate outputs


Prompt: A robot may not injure a human being
Output:  or, through inaction, allow a human being to come to harm.
A robot must obey the orders given it by human beings except where such orders would conflict with the First Law.
A robot must protect its own existence as long as such protection does not conflict with the First or Second Law.
The Three Laws of Robotics were created by Isaac Asimov in 1942. They are the foundation of robotics and artificial intelligence.
The Three Laws of Robotics are the foundation of robotics and artificial intelligence. They were created by Isaac Asimov in 194

Prompt: To be or not to be,
Output:  that is the question.
I am not sure what I would do if I had to make a decision to live or die.
I would probably die, just to be sure.
I think I would die, because if I were to live, I would not be able to live.
If I were dead, I could live, but I would not be happy.
If I had to choose between living and dying, I think that I would choose to live, because if I 